In [37]:
import nba_api

In [19]:
# scripts/fetch_top30_player_stats.py

import json
import os
from tqdm import tqdm
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from requests.exceptions import ReadTimeout

# 1) Choose your Top-30 active stars
TARGET_PLAYERS = [
    "LeBron James","Stephen Curry","Kevin Durant","Giannis Antetokounmpo",
    "Kawhi Leonard","James Harden","Nikola Jokic","Luka Doncic",
    "Damian Lillard","Joel Embiid","Anthony Davis","Kyrie Irving",
    "Chris Paul","Russell Westbrook","Paul George","Klay Thompson",
    "Jimmy Butler","Draymond Green","Devin Booker","Jayson Tatum",
    "Ben Simmons","Karl-Anthony Towns","Bradley Beal","DeMar DeRozan",
    "Zach LaVine","Ja Morant","Trae Young","Donovan Mitchell",
    "Jaylen Brown","Shai Gilgeous-Alexander"
]

# 2) Paths
OUTPUT_STATS = os.path.join("public", "data", "player_stats.json")
os.makedirs(os.path.dirname(OUTPUT_STATS), exist_ok=True)

all_stats = {}

print(f"Fetching stats for Top {len(TARGET_PLAYERS)} players...\n")

for name in tqdm(TARGET_PLAYERS, desc="Players"):
    # 3) Lookup player ID
    matches = players.find_players_by_full_name(name)
    if not matches:
        print(f"⚠️  No match for '{name}', skipping.")
        continue
    pid = matches[0]["id"]

    # 4) Fetch profile info
    try:
        info_df = commonplayerinfo.CommonPlayerInfo(player_id=pid).get_data_frames()[0]
        row = info_df.loc[0, [
            "DISPLAY_FIRST_LAST","TEAM_ABBREVIATION","POSITION","HEIGHT","WEIGHT"
        ]]
        profile = {
            "DISPLAY_FIRST_LAST": row["DISPLAY_FIRST_LAST"],
            "TEAM_ABBREVIATION": row["TEAM_ABBREVIATION"],
            "POSITION": row["POSITION"],
            "HEIGHT": row["HEIGHT"],
            "WEIGHT": row["WEIGHT"],
        }
    except Exception as e:
        print(f"  → Profile error for {name}: {e}")
        continue

    # 5) Fetch career stats via get_dict() with retry
    season_stats = {
        "SEASON": "", "GP": 0, "PTS": 0, "REB": 0,
        "AST": 0, "FG_PCT": 0.0, "FG3_PCT": 0.0, "FT_PCT": 0.0
    }
    for attempt in range(1, 4):
        try:
            career = playercareerstats.PlayerCareerStats(player_id=pid)
            career_dict = career.get_dict()
            rs = career_dict["resultSets"][0]
            headers = rs["headers"]
            rows    = rs["rowSet"]
            if not rows:
                raise ValueError("no career rows")
            latest = rows[-1]
            data  = dict(zip(headers, latest))
            season_stats = {
                "SEASON": data.get("SEASON_ID", ""),
                "GP": int(data.get("GP", 0)),
                "PTS": int(data.get("PTS", 0)),
                "REB": int(data.get("REB", 0)),
                "AST": int(data.get("AST", 0)),
                "FG_PCT": float(data.get("FG_PCT", 0.0) * 100),
                "FG3_PCT": float(data.get("FG3_PCT", 0.0) * 100),
                "FT_PCT": float(data.get("FT_PCT", 0.0) * 100),
            }
            break
        except ReadTimeout:
            print(f"    → Timeout fetching careers for {name}, retry {attempt}")
        except Exception as e:
            print(f"    → Error fetching careers for {name}: {e}")
            break

    # 6) Combine and record
    all_stats[name] = { **profile, **season_stats }

# 7) Write out JSON
with open(OUTPUT_STATS, "w") as f:
    json.dump(all_stats, f, indent=2)

print(f"\n✅  Saved stats for {len(all_stats)} players to {OUTPUT_STATS}")


Fetching stats for Top 30 players...



Players:  30%|██████████████████████▏                                                   | 9/30 [00:34<03:25,  9.78s/it]

  → Profile error for Damian Lillard: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


Players:  70%|███████████████████████████████████████████████████                      | 21/30 [00:39<00:04,  1.92it/s]

    → Timeout fetching careers for Karl-Anthony Towns, retry 1
    → Timeout fetching careers for Karl-Anthony Towns, retry 2


...

  → Profile error for Donovan Mitchell: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


Players:  97%|██████████████████████████████████████████████████████████████████████▌  | 29/30 [05:42<00:30, 30.03s/it]

  → Profile error for Jaylen Brown: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


Players: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [06:12<00:00, 12.42s/it]

  → Profile error for Shai Gilgeous-Alexander: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

✅  Saved stats for 21 players to public\data\player_stats.json


In [38]:
# scripts/fetch_upcoming_games.py

import os
import json
import time
import datetime

from nba_api.stats.endpoints import ScoreboardV2
from nba_api.stats.static import teams
from requests.exceptions import ConnectionError, ReadTimeout
from http.client import RemoteDisconnected

# Output path
OUT_PATH = os.path.join("public", "data", "upcoming_games.json")
os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

# Build TEAM_ID → abbreviation map
team_list = teams.get_teams()
id_to_abbrev = {t["id"]: t["abbreviation"] for t in team_list}

games_out = []
today = datetime.date.today()

# Try each of the next 3 days
for offset in range(3):
    date = today + datetime.timedelta(days=offset)
    date_str = date.strftime("%m/%d/%Y")

    # retry up to 3 times
    for attempt in range(1, 4):
        try:
            sb = ScoreboardV2(game_date=date_str)
            df = sb.get_data_frames()[0]  # one row per game
            break
        except (ConnectionError, ReadTimeout, RemoteDisconnected) as e:
            print(f"[{date_str}] Attempt {attempt} failed: {e}")
            if attempt < 3:
                time.sleep(1)
            else:
                print(f"[{date_str}] Skipping after 3 failed attempts.")
                df = None
    if df is None:
        continue

    # build output entries
    for row in df.itertuples():
        games_out.append({
            "game_id":            row.GAME_ID,
            "date":               row.GAME_DATE_EST,
            "time":               row.GAME_STATUS_TEXT,
            "visitor_team":       id_to_abbrev.get(row.VISITOR_TEAM_ID, ""),
            "home_team":          id_to_abbrev.get(row.HOME_TEAM_ID, ""),
            "arena":              row.ARENA_NAME,
            "broadcaster":        row.NATL_TV_BROADCASTER_ABBREVIATION,
            "live_period":        row.LIVE_PERIOD,
            "live_period_bcast":  row.LIVE_PERIOD_TIME_BCAST,
        })

# Write JSON
with open(OUT_PATH, "w") as f:
    json.dump(games_out, f, indent=2)

print(f"Wrote {len(games_out)} upcoming games to {OUT_PATH}")


Wrote 5 upcoming games to public\data\upcoming_games.json


In [34]:
df.to_json(orient='records')

'[{"GAME_DATE_EST":"2025-05-07T00:00:00","GAME_SEQUENCE":1,"GAME_ID":"0042400212","GAME_STATUS_ID":1,"GAME_STATUS_TEXT":"7:00 pm ET","GAMECODE":"20250507\\/NYKBOS","HOME_TEAM_ID":1610612738,"VISITOR_TEAM_ID":1610612752,"SEASON":"2024","LIVE_PERIOD":0,"LIVE_PC_TIME":"     ","NATL_TV_BROADCASTER_ABBREVIATION":"TNT\\/truTV\\/Max","HOME_TV_BROADCASTER_ABBREVIATION":null,"AWAY_TV_BROADCASTER_ABBREVIATION":null,"LIVE_PERIOD_TIME_BCAST":"Q0       - TNT\\/truTV\\/Max","ARENA_NAME":"TD Garden","WH_STATUS":0,"WNBA_COMMISSIONER_FLAG":0},{"GAME_DATE_EST":"2025-05-07T00:00:00","GAME_SEQUENCE":2,"GAME_ID":"0042400222","GAME_STATUS_ID":1,"GAME_STATUS_TEXT":"9:30 pm ET","GAMECODE":"20250507\\/DENOKC","HOME_TEAM_ID":1610612760,"VISITOR_TEAM_ID":1610612743,"SEASON":"2024","LIVE_PERIOD":0,"LIVE_PC_TIME":"     ","NATL_TV_BROADCASTER_ABBREVIATION":"TNT\\/truTV\\/Max","HOME_TV_BROADCASTER_ABBREVIATION":null,"AWAY_TV_BROADCASTER_ABBREVIATION":null,"LIVE_PERIOD_TIME_BCAST":"Q0       - TNT\\/truTV\\/Max","AREN

In [17]:
# scripts/fetch_team_records.py

import os
import json
from nba_api.stats.endpoints import LeagueStandings
from nba_api.stats.library.parameters import LeagueID, SeasonTypeAllStar

OUT_PATH = os.path.join("public", "data", "team_records.json")
os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

# Current season (adjust as needed)
SEASON = "2024-25"
standings = LeagueStandings(
    league_id=LeagueID.default,
    season=SEASON,
    season_type_all_star=SeasonTypeAllStar.regular_season
).get_data_frames()[0]

records = []
for _, row in standings.iterrows():
    records.append({
        "abbreviation": row["TeamAbbreviation"],
        "wins": row["WINS"],
        "losses": row["LOSSES"],
        "win_pct": row["WIN_PCT"],
        "conf_rank": row["ConferenceRank"],
    })

with open(OUT_PATH, "w") as f:
    json.dump(records, f, indent=2)

print(f"Wrote {len(records)} team records to {OUT_PATH}")


AttributeError: type object 'SeasonTypeAllStar' has no attribute 'regular_season'

In [13]:
# scripts/fetch_shot_logs.py

import os
import time
from tqdm import tqdm
import pandas as pd

from nba_api.stats.endpoints import ShotChartDetail, CommonTeamRoster
from nba_api.stats.static import teams
from requests.exceptions import ReadTimeout

# 1) Build TEAM_ID → abbreviation map
team_list = teams.get_teams()
id_to_abbrev = {t["id"]: t["abbreviation"] for t in team_list}

SEASON = "2024-25"  # NBA Stats format

# 2) Fetch rosters for each team, with retry logic
all_players = []
print("=== Fetching team rosters ===")
for t in team_list:
    tid = t["id"]
    abbrev = t["abbreviation"]
    for attempt in range(1, 4):
        try:
            print(f"Team {abbrev}: roster attempt {attempt}")
            roster_df = CommonTeamRoster(
                team_id=tid,
                season=SEASON,
                timeout=30
            ).get_data_frames()[0]
            break
        except ReadTimeout:
            print(f"  → Timeout fetching roster for {abbrev}, retrying...")
            time.sleep(1)
        except Exception as e:
            print(f"  → Error fetching roster for {abbrev}: {e}")
            roster_df = pd.DataFrame()
            break
    else:
        print(f"  → Failed to fetch roster for {abbrev} after 3 attempts")
        roster_df = pd.DataFrame()

    if not roster_df.empty:
        for _, row in roster_df.iterrows():
            all_players.append({
                "id": int(row["PLAYER_ID"]),
                "name": row["PLAYER"]
            })

# 3) De-duplicate players by ID
unique_players = list({p["id"]: p for p in all_players}.values())
print(f"Found {len(unique_players)} unique players on 2024-25 rosters.\n")

# 4) Fetch shot chart for each player
all_dfs = []
print("=== Fetching shot charts ===")
for p in tqdm(unique_players, desc="Players"):
    pid = p["id"]
    name = p["name"]
    success = False

    for attempt in range(1, 4):
        try:
            print(f"[{name} ({pid})] attempt {attempt}")
            resp = ShotChartDetail(
                team_id=0,
                player_id=pid,
                season_nullable=SEASON,            # <-- use season_nullable here
                season_type_all_star="Regular Season",
                context_measure_simple="FGA",
                timeout=30
            )
            df = resp.get_data_frames()[0]
            success = True
            break
        except ReadTimeout:
            print(f"  → Timeout for {name}, retrying...")
            time.sleep(1)
        except Exception as e:
            print(f"  → Error fetching data for {name}: {e}")
            break

    if not success:
        print(f"  → Skipping {name} after 3 failed attempts\n")
        continue

    # inject team abbreviation
    df["abbreviation"] = df["TEAM_ID"].map(id_to_abbrev)

    df_sel = df[[
        "PLAYER_NAME",
        "PLAYER_ID",
        "abbreviation",
        "LOC_X",
        "LOC_Y",
        "SHOT_ATTEMPTED_FLAG",
        "SHOT_MADE_FLAG",
    ]].copy()

    df_sel.columns = [
        "player_name",
        "player_id",
        "abbreviation",
        "x",
        "y",
        "shot_attempted",
        "shot_made",
    ]

    all_dfs.append(df_sel)

# 5) Concatenate and write JSON
if all_dfs:
    shots_df = pd.concat(all_dfs, ignore_index=True)
    out_path = os.path.join("public", "data", "all_shot_data.json")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    shots_df.to_json(out_path, orient="records")
    print(f"\nWrote {len(shots_df)} shots to {out_path}")
else:
    print("\nNo shot data to write.")


=== Fetching team rosters ===
Team ATL: roster attempt 1
Team BOS: roster attempt 1
Team CLE: roster attempt 1
Team NOP: roster attempt 1
Team CHI: roster attempt 1
Team DAL: roster attempt 1
Team DEN: roster attempt 1
Team GSW: roster attempt 1
Team HOU: roster attempt 1
Team LAC: roster attempt 1
Team LAL: roster attempt 1
Team MIA: roster attempt 1
Team MIL: roster attempt 1
Team MIN: roster attempt 1
Team BKN: roster attempt 1
Team NYK: roster attempt 1
  → Timeout fetching roster for NYK, retrying...
Team NYK: roster attempt 2
  → Timeout fetching roster for NYK, retrying...
Team NYK: roster attempt 3
  → Timeout fetching roster for NYK, retrying...
  → Failed to fetch roster for NYK after 3 attempts
Team ORL: roster attempt 1
  → Timeout fetching roster for ORL, retrying...
Team ORL: roster attempt 2
  → Timeout fetching roster for ORL, retrying...
Team ORL: roster attempt 3
  → Timeout fetching roster for ORL, retrying...
  → Failed to fetch roster for ORL after 3 attempts
Team 

Players:   0%|                                                                                 | 0/480 [00:00<?, ?it/s]

[Jacob Toppin (1631210)] attempt 1


Players:   0%|▏                                                                        | 1/480 [00:01<09:38,  1.21s/it]

[Jalen Johnson (1630552)] attempt 1


...

[Tidjane Salaün (1642275)] attempt 1


Players: 100%|██████████████████████████████████████████████████████████████████████▊| 479/480 [07:06<00:00,  1.04it/s]

[Taj Gibson (201959)] attempt 1


Players: 100%|███████████████████████████████████████████████████████████████████████| 480/480 [07:06<00:00,  1.12it/s]



Wrote 196189 shots to public\data\all_shot_data.json
